# Predicting China Scholarship additional cost

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('./data/cleaned.csv')

In [4]:
data.head()

,schol_id,University,Major,Level,Language,Tuition Covered,Accomodation covered?,Living Expense Covered?,Tuition fees to pay,Original Tuition fee,Start_Month,Start_Year,Accomodation_To_Pay,Accomodation_duration,Expense_To_Pay,Expense_duration
0,1,Shanghai University of Traditional Chinese Med...,Summer School (TCM),Non-Degree,English,7500.0,0,0,0,7500.0,Jun,2020,9750.0,MONTH,1000.0,MONTH
1,2,Changchun University of Science and Technology,Preparatory Program of Humanities Direction(Pa...,Non-Degree,Chinese,10000.0,0,0,10000,20000.0,Sep,2019,1500.0,MONTH,1500.0,MONTH
2,3,Changchun University of Science and Technology,Preparatory Program of Business Direction(Path A),Non-Degree,Chinese,10000.0,0,0,10000,20000.0,Sep,2019,1500.0,MONTH,1500.0,MONTH
3,4,Changchun University of Science and Technology,Preparatory Program of Science and Engineering...,Non-Degree,Chinese,10000.0,0,0,10000,20000.0,Sep,2019,1500.0,MONTH,1500.0,MONTH
4,5,Qingdao University,Long-term Chinese Language Program,Non-Degree,Chinese,7000.0,1,1,0,7000.0,Sep,2019,0.0,YEAR,0.0,MONTH


In [5]:
data.tail()

,schol_id,University,Major,Level,Language,Tuition Covered,Accomodation covered?,Living Expense Covered?,Tuition fees to pay,Original Tuition fee,Start_Month,Start_Year,Accomodation_To_Pay,Accomodation_duration,Expense_To_Pay,Expense_duration
3571,3601,East China University of Science and Technology,Environmental Science and Engineering,Phd,English,38950.0,1,1,0,38950.0,Sep,2019,0.0,YEAR,0.0,MONTH
3572,3602,East China University of Science and Technology,Power Engineering and Engineering Thermophysics,Phd,English,38950.0,1,1,0,38950.0,Sep,2019,0.0,YEAR,0.0,MONTH
3573,3603,East China University of Science and Technology,Thermal Engineering,Phd,English,38950.0,1,1,0,38950.0,Sep,2019,0.0,YEAR,0.0,MONTH
3574,3604,East China University of Science and Technology,Synthesis of Fine Chemical and Molecular,Phd,English,38950.0,1,1,0,38950.0,Sep,2019,0.0,YEAR,0.0,MONTH
3575,3605,East China University of Science and Technology,Sociology of Law,Phd,English,38950.0,1,1,0,38950.0,Sep,2019,0.0,YEAR,0.0,MONTH


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3576 entries, 0 to 3575
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   schol_id                 3576 non-null   int64  
 1   University               3576 non-null   object 
 2   Major                    3576 non-null   object 
 3   Level                    3576 non-null   object 
 4   Language                 3576 non-null   object 
 5   Tuition Covered          3453 non-null   float64
 6   Accomodation covered?    3576 non-null   int64  
 7   Living Expense Covered?  3576 non-null   int64  
 8   Tuition fees to pay      3576 non-null   int64  
 9   Original Tuition fee     3453 non-null   float64
 10  Start_Month              3576 non-null   object 
 11  Start_Year               3576 non-null   int64  
 12  Accomodation_To_Pay      3576 non-null   float64
 13  Accomodation_duration    3576 non-null   object 
 14  Expense_To_Pay          

In [8]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
schol_id,3576.0,1816.503076,1034.126355,1.0,923.75,1817.5,2711.25,3605.0
Tuition Covered,3453.0,20531.704315,12428.124918,0.0,10625.00,20000.0,26500.00,80000.0
Accomodation covered?,3576.0,0.232942,0.422765,0.0,0.00,0.0,0.00,1.0
Living Expense Covered?,3576.0,0.251119,0.433717,0.0,0.00,0.0,1.00,1.0
Tuition fees to pay,3576.0,8433.516499,9519.340675,0.0,0.00,6000.0,11000.00,69000.0
Original Tuition fee,3453.0,28693.955980,15341.450293,0.0,20000.00,26500.0,34000.00,148000.0
Start_Year,3576.0,2019.056208,0.230355,2019.0,2019.00,2019.0,2019.00,2020.0
Accomodation_To_Pay,3576.0,1376.461969,2802.526970,0.0,400.00,900.0,1200.00,24000.0
Expense_To_Pay,3576.0,1070.987136,601.297528,0.0,1000.00,1000.0,1500.00,2700.0
